In [1]:
import pandas as pd
from IPython.display import Markdown

from summer2 import CompartmentalModel
from summer2.parameters import Parameter, DerivedOutput

In [2]:
# Base model construction
model = CompartmentalModel(
    times=[1850.0, 2024.0],
    compartments=["susceptible", "early latent", "late latent", "infectious", "recovered"],
    infectious_compartments=["infectious"],
)
model.set_initial_population({"susceptible": 1.0, "infectious": 1.0})

In [3]:
# Add latency structures
latency_flows = [
    ("stabilisation", "early latent", "late latent"),
    ("early activation", "early latent", "infectious"),
    ("late activation", "late latent", "infectious"),
]
for flow, source, dest in latency_flows:
    model.add_transition_flow(flow, Parameter(f"{flow} rate"), source, dest)

In [4]:
# Add infection processes
infection_origins = ["susceptible", "late latent", "recovered"]
for origin in infection_origins:
    model.add_infection_frequency_flow(f"infection_from_{origin}", Parameter("contact rate"), origin, "early latent")

In [5]:
# Track relevant outputs
for timing in ["early", "late"]:
    model.request_output_for_flow(f"{timing} activation", f"{timing} activation", save_results=False)
inc = model.request_function_output("incidence", DerivedOutput("early activation") + DerivedOutput("late activation"))
model.request_function_output("noticiations", inc * Parameter("case detection rate"))

DerivedOutput noticiations {'request_type': 'param_func', 'func': Function: 'multiply', args=(DerivedOutput...esults': True}, Parameter case detection rate), kwargs={}), 'save_results': True}

In [6]:
# Parameters
params = {
    "contact rate": 1.0,
    "stabilisation rate": 1.0,
    "early activation rate": 1.0,
    "late activation rate": 1.0,
    "case detection rate": 1.0,
}
param_units = {
    "contact rate": "per day",
    "stabilisation rate": "per day",
    "early activation rate": "per day",
    "late activation rate": "per day",
    "case detection rate": "",
}
fixed_param_table = pd.DataFrame({"values": params, "units": param_units})

In [7]:
Markdown(fixed_param_table.to_markdown())

|                       |   values | units   |
|:----------------------|---------:|:--------|
| contact rate          |        1 | per day |
| stabilisation rate    |        1 | per day |
| early activation rate |        1 | per day |
| late activation rate  |        1 | per day |
| case detection rate   |        1 |         |

In [8]:
model.run(params)

In [9]:
# model.flows